In [456]:
#import libraries
import os
import requests
import json
import numpy as np
import tensorflow as tf
import pandas as pd
from config import api_key
from pprint import pprint
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense
from tensorflow.keras.datasets import mnist
from sklearn.metrics import classification_report
from sklearn.svm import SVC 
import matplotlib.pyplot as plt
import numpy as np


In [457]:
#set up api request
url = "https://play.limitlesstcg.com/api"
api_key = "key="+ api_key
query = "game=PTCG&format=standard&"
tournamentid = ""

In [458]:
#create api request
response = requests.get(url+"/tournaments/"+tournamentid+"?"+query+api_key)

In [459]:
#store api response
tournaments = response.json()
tournaments

[{'game': 'PTCG',
  'name': 'The Discard Pile Weekly #19 (Standard) ',
  'date': '2023-05-31T12:00:00.000Z',
  'format': 'STANDARD',
  'id': '646e56d19b9f5808497a93fc',
  'players': 34},
 {'game': 'PTCG',
  'name': 'Late Night #104 on PTCG Live | Road to Japan',
  'date': '2023-05-30T23:00:00.000Z',
  'format': 'STANDARD',
  'id': '640677ec24ea2b561a98792e',
  'players': 242},
 {'game': 'PTCG',
  'name': 'Late Night #103 on PTCG Live | Road to Japan',
  'date': '2023-05-30T16:00:00.000Z',
  'format': 'STANDARD',
  'id': '6406779924ea2b561a98792c',
  'players': 177},
 {'game': 'PTCG',
  'name': 'Free entry ║ Deck Out Mondays ║ 200 codes',
  'date': '2023-05-30T00:00:00.000Z',
  'format': 'STANDARD',
  'id': '646bf66c9b9f5808497a8464',
  'players': 92},
 {'game': 'PTCG',
  'name': "Sunny's Weekly #105",
  'date': '2023-05-29T22:05:00.000Z',
  'format': 'STANDARD',
  'id': '646c22f09b9f5808497a8618',
  'players': 60},
 {'game': 'PTCG',
  'name': 'FalaMeowth! "5/0 Hero" #11',
  'date': '20

In [460]:
#create empty dataframe and add archetype
cards= []
cards.append("archetype")
carddf = pd.DataFrame([], columns=[cards])
carddf["archetype"].astype(str)
carddf.head()

,archetype


In [461]:
#create empty lists to store output
tournament_names = []
decklists = []
deckarch = []
card_archetypes = []
count=1
#iterate through tournaments, and add values for cards as well as archetype to dataframe
for tournament in tournaments:
    tournament_names.append(tournament["name"])
    tournament_ID = tournament["id"]
    print(url+"/tournaments/"+tournament_ID+"/standings?"+query+api_key)
    print("Tourney Count " + str(count))
    count = count+1
    response = requests.get(url+"/tournaments/"+tournament_ID+"/standings?"+query+api_key)
    data=response.json()
    data
    print(tournament["name"])
    for player in data:
        if player["decklist"] is not None:
            if player["deck"]["id"] != "other":
                idx = carddf.index.tolist() + ['end']
                carddf = carddf.reindex(idx, fill_value=0).reset_index(drop=True)
                if "pokemon" in player["decklist"]:
                    for card in player["decklist"]["pokemon"]:
                        poke_cardnumber=card["set"]+card["number"]
                        carddf.loc[len(carddf.index)-1, poke_cardnumber] = card["count"]
                if "trainer" in player["decklist"]:
                    for card in player["decklist"]["trainer"]:
                        poke_cardnumber=card["set"]+card["number"]
                        carddf.loc[len(carddf.index)-1, poke_cardnumber] = card["count"]      
                
                if "energy" in player["decklist"]:
                    for card in player["decklist"]["energy"]:
                        if card["number"].isnumeric():
                            poke_cardnumber=card["set"]+card["number"]
                            carddf.loc[len(carddf.index)-1, poke_cardnumber] = card["count"]    
                        else:  
                            poke_cardnumber="energy"+card["number"]
                            carddf.loc[len(carddf.index)-1, poke_cardnumber] = card["count"]
                carddf.loc[len(carddf.index)-1, "archetype"] = player["deck"]["id"]
                if player["deck"]["id"] not in card_archetypes:
                    card_archetypes.append(player["deck"]["id"])
carddf=carddf.fillna(0)



https://play.limitlesstcg.com/api/tournaments/646e56d19b9f5808497a93fc/standings?game=PTCG&format=standard&key=5ff01485ba33aa465c6d4bdc23d94e81
Tourney Count 1
The Discard Pile Weekly #19 (Standard) 
https://play.limitlesstcg.com/api/tournaments/640677ec24ea2b561a98792e/standings?game=PTCG&format=standard&key=5ff01485ba33aa465c6d4bdc23d94e81
Tourney Count 2
Late Night #104 on PTCG Live | Road to Japan
https://play.limitlesstcg.com/api/tournaments/6406779924ea2b561a98792c/standings?game=PTCG&format=standard&key=5ff01485ba33aa465c6d4bdc23d94e81
Tourney Count 3
Late Night #103 on PTCG Live | Road to Japan
https://play.limitlesstcg.com/api/tournaments/646bf66c9b9f5808497a8464/standings?game=PTCG&format=standard&key=5ff01485ba33aa465c6d4bdc23d94e81
Tourney Count 4
Free entry ║ Deck Out Mondays ║ 200 codes
https://play.limitlesstcg.com/api/tournaments/646c22f09b9f5808497a8618/standings?game=PTCG&format=standard&key=5ff01485ba33aa465c6d4bdc23d94e81
Tourney Count 5
Sunny's Weekly #105
https://

In [462]:
display(carddf.head())

display(carddf.tail())

,archetype,SIT138,SIT139,SIT147,BST97,BRS40,BST85,BST86,FST175,SIT16,...,SVI10,CRE83,BST31,SHF20,LOR36,BST33,LOR37,CRZ29,LOR34,LOR40
0,lugia-archeops,4.0,3.0,4.0,2.0,2.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,gardevoir-ex-sv,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,arceus-giratina,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,lugia-archeops,4.0,3.0,4.0,2.0,2.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,lugia-archeops,3.0,2.0,4.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


,archetype,SIT138,SIT139,SIT147,BST97,BRS40,BST85,BST86,FST175,SIT16,...,SVI10,CRE83,BST31,SHF20,LOR36,BST33,LOR37,CRZ29,LOR34,LOR40
2318,lugia-archeops,3.0,3.0,4.0,1.0,2.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2319,miraidon-regieleki,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2320,dialga-magnezone,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2321,zeraora-v,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2322,lost-zone-box,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [463]:
#set target/dependent variable
target = carddf["archetype"]
target_names = card_archetypes

In [464]:
# Get the features/set up independent variable
data = carddf.drop("archetype", axis=1)
feature_names = data.columns
data.head()

,SIT138,SIT139,SIT147,BST97,BRS40,BST85,BST86,FST175,SIT16,EVS76,...,SVI10,CRE83,BST31,SHF20,LOR36,BST33,LOR37,CRZ29,LOR34,LOR40
0,4.0,3.0,4.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4.0,3.0,4.0,2.0,2.0,1.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,3.0,2.0,4.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [465]:
# Split Training and Test Data
X_train, X_test, y_train, y_test = train_test_split(data, target, random_state=42)

# Support vector machine linear classifier
model = SVC(kernel='linear')
model.fit(X_train, y_train.values.ravel())

SVC(kernel='linear')

In [466]:
# Training Accuracy
print(f"Training Data Score: {model.score(X_train, y_train)}")
# Model Accuracy
print('Test Acc: %.3f' % model.score(X_test, y_test))

from sklearn import metrics
# Calculate the classification report
predictions = model.predict(X_test)
modelreport= metrics.classification_report(y_test, predictions,
                            labels=target_names)
print(modelreport)

Training Data Score: 1.0
Test Acc: 0.974
                           precision    recall  f1-score   support

           lugia-archeops       0.98      1.00      0.99        48
          gardevoir-ex-sv       1.00      1.00      1.00        64
          arceus-giratina       1.00      1.00      1.00        41
             mew-genesect       1.00      0.95      0.98        21
             mew-meloetta       0.96      1.00      0.98        26
            goodra-lz-box       1.00      1.00      1.00        21
         gardevoir-mewtwo       1.00      1.00      1.00        17
 arceus-ice-rider-calyrex       1.00      1.00      1.00         1
             palkia-vstar       1.00      1.00      1.00        10
            lost-zone-box       0.99      1.00      1.00       103
             arceus-vstar       0.92      0.92      0.92        12
         miraidon-flaaffy       0.79      0.96      0.87        24
             lunarock-pgo       1.00      1.00      1.00         3
         arceus-dura

c:\Users\d_hin\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\d_hin\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\d_hin\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\d_hin\anaconda3\lib\site-packages\sklearn\metrics\_cla

In [468]:
# modeldf=pd.DataFrame(modelreport)


pd.DataFrame(report_dict)


List of tournaments not included in training/test dataset

{'game': 'PTCG', 'name': 'Just Chilling Games #7 Free', 'date': '2023-05-13T18:05:00.000Z', 'format': 'STANDARD', 'id': '645ee7439b9f5808497a4938', 'players': 9}, {'game': 'PTCG', 'name': 'Card Cavern Trading Cards 2023 🏆 Standard #3', 'date': '2023-05-13T16:00:00.000Z', 'format': 'STANDARD', 'id': '643c57300ed4a405d8bd63d1', 'players': 78}, {'game': 'PTCG', 'name': 'The Clara & Jelena Charity Championship', 'date': '2023-05-13T14:15:00.000Z', 'format': 'STANDARD', 'id': '645efd669b9f5808497a4a18', 'players': 17}, {'game': 'PTCG', 'name': "Rossi's Single prize tournament #1", 'date': '2023-05-13T03:00:00.000Z', 'format': 'STANDARD', 'id': '645ebb2f9b9f5808497a4760', 'players': 3}, {'game': 'PTCG', 'name': 'TOURNAMENT OF DOOM! ZELDA! ZELDA! ZELDA!', 'date': '2023-05-13T02:05:00.000Z', 'format': 'STANDARD', 'id': '64484e520ed4a405d8bda0c2', 'players': 68}, {'game': 'PTCG', 'name': 'TanoCup #29', 'date': '2023-05-13T01:12:00.000Z', 'format': 'STANDARD', 'id': '645ee36a9b9f5808497a48e3', 'players': 17}, {'game': 'PTCG', 'name': 'Hoarders Casual #34', 'date': '2023-05-13T00:00:00.000Z', 'format': 'STANDARD', 'id': '6459920e9b9f5808497a30c0', 'players': 43}, {'game': 'PTCG', 'name': 'E&Tc Pokémon LIVE tournament', 'date': '2023-05-12T23:15:00.000Z', 'format': 'STANDARD', 'id': '645ac40a9b9f5808497a3712', 'players': 33}, {'game': 'PTCG', 'name': 'Just Chilling Games #6 Free', 'date': '2023-05-12T23:00:00.000Z', 'format': 'STANDARD', 'id': '645e7b019b9f5808497a4667', 'players': 10}, {'game': 'PTCG', 'name': 'Single Prize Scuffle!', 'date': '2023-05-12T19:00:00.000Z', 'format': 'STANDARD', 'id': '645cca5b9b9f5808497a4025', 'players': 24}

In [469]:
#creat a test query/response
test_tournament_id = "64484e520ed4a405d8bda0c2"
testresponse = requests.get(url+"/tournaments/"+test_tournament_id+"/standings?"+api_key)
testdata=testresponse.json()

testcards= []
testcards.append("archetype")
testcarddf = pd.DataFrame([], columns=[testcards])
testcarddf["archetype"].astype(str)
testcarddf.head()

testcard_archetypes = []
test_tournament_names = []
decklists = []
deckarch = []

#create testing dataframe
testcarddf = pd.DataFrame(columns=carddf.columns)
testdata
for player in testdata:
    if player["decklist"] is not None:
        if player["deck"]["id"] != "other":
            idx = testcarddf.index.tolist() + ['end']
            testcarddf = testcarddf.reindex(idx, fill_value=0).reset_index(drop=True)
            if "pokemon" in player["decklist"]:
                for card in player["decklist"]["pokemon"]:
                    poke_cardnumber=card["set"]+card["number"]
                    testcarddf.loc[len(testcarddf.index)-1, poke_cardnumber] = card["count"]
            if "trainer" in player["decklist"]:
                for card in player["decklist"]["trainer"]:
                    poke_cardnumber=card["set"]+card["number"]
                    testcarddf.loc[len(testcarddf.index)-1, poke_cardnumber] = card["count"]      
            
            if "energy" in player["decklist"]:
                for card in player["decklist"]["energy"]:
                    if card["number"].isnumeric():
                        poke_cardnumber=card["set"]+card["number"]
                        testcarddf.loc[len(testcarddf.index)-1, poke_cardnumber] = card["count"]    
                    else:  
                        poke_cardnumber="energy"+card["number"]
                        testcarddf.loc[len(testcarddf.index)-1, poke_cardnumber] = card["count"]
            testcarddf.loc[len(testcarddf.index)-1, "archetype"] = player["deck"]["id"]
            if player["deck"]["id"] not in card_archetypes:
                testcard_archetypes.append(player["deck"]["id"])
testcarddf=testcarddf.fillna(0)
testcarddf

X_test=testcarddf.drop(columns=["archetype"])


In [470]:

#fit test dataframe to model outputs
column_list = carddf.columns.tolist()
clean_column_list = [*list(zip(*column_list))[0],]
testcarddf2 = testcarddf[clean_column_list]
X_test2=testcarddf2.drop(testcarddf2["archetype"], axis=1)
y_test2=testcarddf2["archetype"]


In [471]:

# Model Accuracy
print('Test Acc: %.3f' % model.score(X_test2, y_test2))

# Calculate the classification report
predictions = model.predict(X_test2)
print(classification_report(y_test2, predictions,
                            labels=target_names))



Test Acc: 0.955
                           precision    recall  f1-score   support

           lugia-archeops       1.00      1.00      1.00         5
          gardevoir-ex-sv       1.00      1.00      1.00         7
          arceus-giratina       1.00      1.00      1.00         5
             mew-genesect       1.00      1.00      1.00         5
             mew-meloetta       1.00      1.00      1.00         1
            goodra-lz-box       1.00      1.00      1.00         3
         gardevoir-mewtwo       1.00      1.00      1.00         2
 arceus-ice-rider-calyrex       0.00      0.00      0.00         0
             palkia-vstar       1.00      1.00      1.00         2
            lost-zone-box       1.00      1.00      1.00        10
             arceus-vstar       0.00      0.00      0.00         0
         miraidon-flaaffy       0.67      1.00      0.80         2
             lunarock-pgo       1.00      1.00      1.00         3
         arceus-duraludon       1.00      1.0

c:\Users\d_hin\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\d_hin\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\d_hin\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\d_hin\anaconda3\lib\site-packages\sklearn\metrics\_cla

In [472]:
print(f"Training Data Score: {model.score(X_train, y_train)}")
print(f"Testing Data Score: {model.score(X_test2, y_test2)}")

Training Data Score: 1.0
Testing Data Score: 0.9545454545454546
